In [1]:
from mobility_service_model import MobilityModel
from activity_scheduler import ActivityScheduler
from mode_choice_nhts import NhtsModeLogit
from two_stage_logit_hlc import TwoStageLogitHLC
from cs_handler import CS_Handler

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from copy import deepcopy
import pickle
import numpy as np
from functools import partial

In [2]:
this_model=MobilityModel('corktown', 'Detroit')

this_model.assign_activity_scheduler(ActivityScheduler(model=this_model))

mode_choice_model=NhtsModeLogit(table_name='corktown', city_folder='Detroit')
this_model.assign_mode_choice_model(mode_choice_model)

this_model.assign_home_location_choice_model(
        TwoStageLogitHLC(table_name='corktown', city_folder='Detroit', 
                         geogrid=this_model.geogrid, 
                         base_vacant_houses=this_model.pop.base_vacant))

for person in this_model.pop.base_sim:
    this_model.activity_scheduler.sample_activity_schedules(person, model=this_model)
    
this_model.create_trips(this_model.pop.base_sim)

Building transport network
Building geography
Building synthetic population
	 Creating Trips


Save the model to save time. Persons already have activities so there are no randomness here

In [2]:
# pickle.dump(this_model, open('saved_model.p', 'wb'))
this_model = pickle.load(open('saved_model.p', 'rb'))

# Tweak ASCs

## Find Best ASCs 

In [34]:
trial_model = deepcopy(this_model)

# object function: returns the MAE between predicted and observed (as object) mode shares
def fobjective(params, object_share, weights=[1,1,1,1]):
    logit_params_tweak = {'ASC for cycle': params['ASC_cycle'], 
                          'ASC for walk': params['ASC_walk'],
                          'ASC for PT': params['ASC_PT']
                         }
    trial_model.mode_choice_model.set_logit_model_params(logit_params_tweak)
    trial_model.predict_trip_modes(trial_model.pop.base_sim)
    mode_split = trial_model.get_mode_split(trial_model.pop.base_sim)
    mode_split = list(mode_split.values())
    loss = np.sum([wei * abs(pred-obj) for pred,obj,wei in zip(mode_split, object_share, weights)]) / np.sum(weights)
    if any([abs(ele)<1e-10 for ele in mode_split]): loss += 1000
    return {'loss': loss, 'status': STATUS_OK}

# set search space for each ASC as a normal distribution with mean=original value, PT using larger std
crt_logit_params = trial_model.mode_choice_model.logit_model['params']
crt_ASC_cycle = crt_logit_params['ASC for cycle']
crt_ASC_walk = crt_logit_params['ASC for walk']
crt_ASC_PT = crt_logit_params['ASC for PT']
fspace = {
    'ASC_cycle': hp.normal('ASC_cycle', crt_ASC_cycle, 3),
    'ASC_walk': hp.normal('ASC_walk', crt_ASC_walk, 3),
    'ASC_PT':hp.normal('ASC_PT', crt_ASC_PT, 5)
}

trials = Trials()
object_share = [0.916, 0.011, 0.066, 0.007]  #observed share to be fitted

# fit object_share for 100 iterations
best = fmin(fn = partial(fobjective, object_share=object_share), 
            space=fspace, algo=tpe.suggest, max_evals=100, trials=trials)

	 Predicting Trip modes                                                                                                
[True, 'NHB']                                                                                                          
[False, 'NHB']                                                                                                         
[True, 'NHB']                                                                                                          
[True, 'NHB']                                                                                                          
[True, 'NHB']                                                                                                          
	 Predicting Trip modes                                                                                                
[True, 'NHB']                                                                                                          
[False, 'NHB']                          

[False, 'NHB']                                                                                                         
[True, 'NHB']                                                                                                          
[True, 'NHB']                                                                                                          
[True, 'NHB']                                                                                                          
	 Predicting Trip modes                                                                                                
[True, 'NHB']                                                                                                          
[False, 'NHB']                                                                                                         
[True, 'NHB']                                                                                                          
[True, 'NHB']                           

[True, 'NHB']                                                                                                          
[True, 'NHB']                                                                                                          
	 Predicting Trip modes                                                                                                
[True, 'NHB']                                                                                                          
[False, 'NHB']                                                                                                         
[True, 'NHB']                                                                                                          
[True, 'NHB']                                                                                                          
[True, 'NHB']                                                                                                          
	 Predicting Trip modes                 

	 Predicting Trip modes                                                                                                
[True, 'NHB']                                                                                                          
[False, 'NHB']                                                                                                         
[True, 'NHB']                                                                                                          
[True, 'NHB']                                                                                                          
[True, 'NHB']                                                                                                          
	 Predicting Trip modes                                                                                                
[True, 'NHB']                                                                                                          
[False, 'NHB']                          

[False, 'NHB']                                                                                                         
[True, 'NHB']                                                                                                          
[True, 'NHB']                                                                                                          
[True, 'NHB']                                                                                                          
	 Predicting Trip modes                                                                                                
[True, 'NHB']                                                                                                          
[False, 'NHB']                                                                                                         
[True, 'NHB']                                                                                                          
[True, 'NHB']                           

[True, 'NHB']                                                                                                          
[True, 'NHB']                                                                                                          
	 Predicting Trip modes                                                                                                
[True, 'NHB']                                                                                                          
[False, 'NHB']                                                                                                         
[True, 'NHB']                                                                                                          
[True, 'NHB']                                                                                                          
[True, 'NHB']                                                                                                          
	 Predicting Trip modes                 

	 Predicting Trip modes                                                                                                
[True, 'NHB']                                                                                                          
[False, 'NHB']                                                                                                         
[True, 'NHB']                                                                                                          
[True, 'NHB']                                                                                                          
[True, 'NHB']                                                                                                          
	 Predicting Trip modes                                                                                                
[True, 'NHB']                                                                                                          
[False, 'NHB']                          

[False, 'NHB']                                                                                                         
[True, 'NHB']                                                                                                          
[True, 'NHB']                                                                                                          
[True, 'NHB']                                                                                                          
	 Predicting Trip modes                                                                                                
[True, 'NHB']                                                                                                          
[False, 'NHB']                                                                                                         
[True, 'NHB']                                                                                                          
[True, 'NHB']                           

[True, 'NHB']                                                                                                          
[True, 'NHB']                                                                                                          
	 Predicting Trip modes                                                                                                
[True, 'NHB']                                                                                                          
[False, 'NHB']                                                                                                         
[True, 'NHB']                                                                                                          
[True, 'NHB']                                                                                                          
[True, 'NHB']                                                                                                          
	 Predicting Trip modes                 

## Check Performance of Best Fit 

In [37]:
print('best ASCs:\n{}\n'.format(best))
trial_model = deepcopy(this_model)
logit_params_tweak = {'ASC for cycle': best['ASC_cycle'], 
                      'ASC for walk': best['ASC_walk'],
                      'ASC for PT': best['ASC_PT']
                     }
trial_model.mode_choice_model.set_logit_model_params(logit_params_tweak)
trial_model.predict_trip_modes(trial_model.pop.base_sim)
mode_split = trial_model.get_mode_split(trial_model.pop.base_sim)
print('\nMode split based on best ASCs:\n{}\n'.format(mode_split))
loss_mode_wise = [abs(pred-obj) for pred,obj in zip(list(mode_split.values()), object_share)]
print('Loss: ', sum(loss_mode_wise)/4)

best ASCs:
{'ASC_PT': -2.437367649219388, 'ASC_cycle': -2.017621754658881, 'ASC_walk': 2.4977466332979517}

	 Predicting Trip modes
[True, 'NHB']
[False, 'NHB']
[True, 'NHB']
[True, 'NHB']
[True, 'NHB']

Mode split based on best ASCs:
{'driving': 0.9174496644295302, 'cycling': 0.011409395973154362, 'walking': 0.06510067114093959, 'pt': 0.006040268456375839}

Loss:  0.0009295302013422853


# Nest Specifications

In [15]:
bikeshare_spec={'name': 'bikeshare', 
                 'attrs':{'active_time_minutes': 'c*1',
                          'cost':1.5},  
                 'copy': 'cycle',
                 "copy_route": "cycling","activity": "cycling","speed_m_s": 4.167,
                 "co2_emissions_kg_met": 0,"fixed_costs": {},
                 'internal_net': 'drive'}
shuttle_spec={'name': 'shuttle', 
                 'attrs':{'vehicle_time_minutes': 'd*1',
                          'cost':1.5},  
                 'copy': 'PT',
                 "copy_route": "driving","activity": "pt","speed_m_s": 8.33,
                 "co2_emissions_kg_met": 0.000066,"fixed_costs": {},
                 'internal_net': 'drive'}

mocho_model = deepcopy(this_model.mode_choice_model)
mocho_model.logit_model['params']['ASC for PT'] = -1.5

# copy from mobility_service_model.py -> MobilityModel -> predict_trip_modes() to add feature_df
all_trips=[]
persons = this_model.pop.base_sim
for person_id, p in enumerate(persons):
    all_trips.extend(p.trips_to_list(person_id=person_id))
mocho_model.generate_feature_df(all_trips)
mocho_model.feature_df['drive_vehicle_time_minutes'] = mocho_model.feature_df['drive_vehicle_time_minutes']*3/4
mocho_model.base_feature_df = mocho_model.feature_df

[True, 'NHB']
[False, 'NHB']
[True, 'NHB']
[True, 'NHB']
[True, 'NHB']


## Fit Params 

In [19]:
base_mnl_model = deepcopy(mocho_model)
base_mnl_model.mnl_predict()

# object function: returns the MAE between predicted and object substitution share
# substitution share is calculated as total decrease of probabilities of drive, cycle, walk, and PT from base scenario
# to new modes scenario.
def fobjective_nest(params, object_subs, weights=[1,1,1,1], disp=False):
    lambda_PT, lambda_walk, ASC_similarity_PT = params['lambda_PT'], params['lambda_walk'], params['ASC_similarity_PT']
    new_gnl_model = deepcopy(mocho_model)
    new_gnl_model.set_new_alt(bikeshare_spec)
    new_gnl_model.set_new_alt(shuttle_spec)
    crt_logit_params = new_gnl_model.logit_model['params']
    params_for_bikeshare = {}
    for g_attr in new_gnl_model.logit_generic_attrs:
        params_for_bikeshare['{} for bikeshare'.format(g_attr)] = \
            crt_logit_params['{} for PT'.format(g_attr)] * ASC_similarity_PT + \
            crt_logit_params['{} for walk'.format(g_attr)] * (1-ASC_similarity_PT)
    params_for_bikeshare['ASC for bikeshare'] = crt_logit_params['ASC for PT'] * ASC_similarity_PT + \
            crt_logit_params['ASC for walk'] * (1-ASC_similarity_PT)
    new_gnl_model.set_logit_model_params(params_for_bikeshare)
    nests_spec=[{'name': 'PT_like', 'alts':['bikeshare', 'PT', 'shuttle'], 'lambda':lambda_PT},
                {'name': 'walk_like', 'alts':['bikeshare','walk'], 'lambda':lambda_walk}
               ]
    new_gnl_model.gnl_predict(nests_spec)
    substitution = np.zeros(4)
    for base_prob_row, gnl_prob_row in zip(base_mnl_model.predicted_prob, new_gnl_model.predicted_prob):
        substitution += base_prob_row - gnl_prob_row[:4]
    substitution  = substitution / substitution.sum()
    if disp:
        tmp = {mode: value for mode, value in zip(['drive','cycle','walk','PT'], substitution)}
        print('\nSubstitution from: \n{}'.format(tmp))
    loss = np.sum([wei * abs(pred-obj) for pred,obj,wei in zip(substitution, object_subs, weights)]) / np.sum(weights)
    return {'loss': loss, 'status': STATUS_OK}

# set search space for lambdas and weight param for ASC beta of bikeshare, all of them are 0~1.
fspace = {
    'lambda_PT': hp.uniform('lambda_PT', 0.05, 0.95),
    'lambda_walk': hp.uniform('lambda_walk', 0.05, 0.95),
    'ASC_similarity_PT':hp.uniform('ASC_similarity_PT', 0, 1)
}

trials = Trials()
object_subs = [0.1, 0.12, 0.3, 0.48]

# fit for 100 iterations
best = fmin(fn = partial(fobjective_nest, object_subs=object_subs, weights=[1,1,3,3]), 
            space=fspace, algo=tpe.suggest, max_evals=100, trials=trials)

[warning] no information for bikeshare_vehicle_time_minutes, set to 0                                                  
[warning] no information for shuttle_active_time_minutes, set to 0                                                     
[warning] no information for bikeshare_vehicle_time_minutes, set to 0                                                  
[warning] no information for shuttle_active_time_minutes, set to 0                                                     
[warning] no information for bikeshare_vehicle_time_minutes, set to 0                                                  
[warning] no information for shuttle_active_time_minutes, set to 0                                                     
[warning] no information for bikeshare_vehicle_time_minutes, set to 0                                                  
[warning] no information for shuttle_active_time_minutes, set to 0                                                     
[warning] no information for bikeshare_v

[warning] no information for bikeshare_vehicle_time_minutes, set to 0                                                  
[warning] no information for shuttle_active_time_minutes, set to 0                                                     
[warning] no information for bikeshare_vehicle_time_minutes, set to 0                                                  
[warning] no information for shuttle_active_time_minutes, set to 0                                                     
[warning] no information for bikeshare_vehicle_time_minutes, set to 0                                                  
[warning] no information for shuttle_active_time_minutes, set to 0                                                     
[warning] no information for bikeshare_vehicle_time_minutes, set to 0                                                  
[warning] no information for shuttle_active_time_minutes, set to 0                                                     
[warning] no information for bikeshare_v

[warning] no information for bikeshare_vehicle_time_minutes, set to 0                                                  
[warning] no information for shuttle_active_time_minutes, set to 0                                                     
[warning] no information for bikeshare_vehicle_time_minutes, set to 0                                                  
[warning] no information for shuttle_active_time_minutes, set to 0                                                     
[warning] no information for bikeshare_vehicle_time_minutes, set to 0                                                  
[warning] no information for shuttle_active_time_minutes, set to 0                                                     
[warning] no information for bikeshare_vehicle_time_minutes, set to 0                                                  
[warning] no information for shuttle_active_time_minutes, set to 0                                                     
[warning] no information for bikeshare_v

In [20]:
print('best params:\n{}\n'.format(best))
results = fobjective_nest(best, object_subs=object_subs, disp=True)
print(results)

best params:
{'ASC_similarity_PT': 0.5332377487023906, 'lambda_PT': 0.07966770149124669, 'lambda_walk': 0.15423621977633595}

[warning] no information for bikeshare_vehicle_time_minutes, set to 0
[warning] no information for shuttle_active_time_minutes, set to 0

Substitution from: 
{'drive': 0.39456307708187655, 'cycle': 0.013680556531568787, 'walk': 0.11455749352380466, 'PT': 0.4771988728627499}
{'loss': 0.14728153854093828, 'status': 'ok'}
